### Check for errors

In [18]:
import src.utils as utils
from src.solvers import pg_solver
from ray import tune

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

from tbparse import SummaryReader
import seaborn as sns; sns.set_theme()

In [19]:
# Load best results from hyperparameter tuning
#exp_path = 'hypersearch2/pg_solver/0020/0020/rand_n=0020_k=03_m=0020_i=01/arch'
exp_path = 'hypersearch/pg_solver/0010/0010/rand_n=0010_k=03_m=0010_i=01/arch'
restored_tuner = tune.Tuner.restore(path=exp_path,
                                    trainable=pg_solver,
                                    resume_unfinished=False,
                                    resume_errored=False,
                                    restart_errored=False)
results = restored_tuner.get_results()

# Check if there have been any errors
print(results.errors)
print(results.num_errors)
print(results.num_terminated)

[]
0
50


In [20]:
num_vars = 10
exp_folder = 'hypersearch/pg_solver'

# Get path for every hyperparameter search
exp_folder = os.path.join(exp_folder, f'{num_vars:04d}')
folder_clauses = sorted([os.path.join(exp_folder, folder) for folder in os.listdir(exp_folder)])
folder_instances = sorted([os.path.join(folder, instance) for folder in folder_clauses for instance in os.listdir(folder)])
folder_assumption = sorted([os.path.join(folder, assumption) for folder in folder_instances for assumption in os.listdir(folder)])

errors = 0
for i, search_path in enumerate(folder_assumption):
    restored_tuner = tune.Tuner.restore(path=exp_path,
                                    trainable=pg_solver,
                                    resume_unfinished=False,
                                    resume_errored=False,
                                    restart_errored=False)
    
    results = restored_tuner.get_results()
    
    if results.num_errors > 0:
        errors += 1
        #print(i, results.errors, results.num_errors, results.num_terminated)
        #print(search_path)
        #print()

print(f'Total errors for n = {num_vars}: {errors}. Total searchs: {len(folder_assumption)}')

Total errors for n = 10: 0. Total searchs: 120


# Results

### Why it is important to load dataframe instead of get_best_results

In [17]:
# Load best results
#exp_path = 'hypersearch_31may/pg_solver/0050/0075/rand_n=0050_k=03_m=0075_i=01/baseline'
exp_path = 'hypersearch/pg_solver/0010/0045/rand_n=0010_k=03_m=0045_i=01/arch'

restored_tuner = tune.Tuner.restore(path=exp_path,
                                    trainable=pg_solver)

results = restored_tuner.get_results()

# Check if there have been errors
if results.errors:
    print("Errors reported.")
else:
    print("No errors reported.")

# Get best trial
print(results.get_best_result(metric='num_sat_eval', mode='max', scope='last').metrics['num_sat_eval'])
print(results.get_best_result(metric='num_sat_eval', mode='max', scope='all').metrics['num_sat_eval'])
print(results.get_dataframe(filter_metric='num_sat_eval', filter_mode='max')['num_sat_eval'].max())

No errors reported.
44.0
43.0
44.0


In [ ]:
# Get the path for every hyperparameters search